# 17. Frame Selected Discriminality and ICC

In [1]:
from src import *

In [2]:
import pingouin

## Description

We evaluate frame selected's discriminality and ICC values. 

GOAL: Compute frame selected's discriminality and ICC values. 

## Table of Contents
- [Frame selected and discrim / ICC](#comparison)

<a class="anchor" id="comparison"></a>
## Frame selected (by emotion) gradients

In [3]:
# Params
embedding  = "dm" # diffusion map
aff_kernel = "pearson" #affinity matrix kernel
align_meth = "procrustes"
n_iter     = 10 # procrustes align number of iteration
nb_comp    = 3
rs         = 99

D = lambda x,y: ((x-y)**2).sum(axis=1)

Load frame selected and full movie gradients (individual level)

In [4]:
total_series = load('./data/fmri_414_series.pkl')
f_series = load('./resources/treshed_series.pkl')

# del fselect_series['Rest']
del total_series['Rest']

In [5]:
fselected_series = f_series[1][(50, 100)]

In [6]:
# compute threshed picked timeseries of same length
f_selected_gradients = {k:{} for k in fselected_series}

for mname in fselected_series:
    tmp_dict = fselected_series[mname]
    
    for _id in tmp_dict:
        tmp_series = tmp_dict[_id]
        fc_selects = FC(tmp_series)
        g_selects = GradientMaps(n_components=nb_comp, approach=embedding, 
                            kernel=aff_kernel, random_state=rs)
        g_selects.fit(fc_selects)
        grad = g_selects.gradients_
        fscore = D(grad.mean(axis=0), grad)
        f_selected_gradients[mname][_id] = fscore

In [7]:
# compute randomly picked timeseries of same length
np.random.seed(19)
full_movie_grads = {k:{} for k in fselected_series}

for mname in fselected_series:
    perc_left = fselected_series[mname]['S15'].shape[0]
    tmp_dict = total_series[mname]
    
    for _id in tmp_dict:
        tmp_series = tmp_dict[_id]
        tmp_index = np.random.choice(np.arange(len(tmp_series)), perc_left,replace=False)
        tmp_series = tmp_series[tmp_index]
        
        fc_selects = FC(tmp_series)
        g_selects = GradientMaps(n_components=nb_comp, approach=embedding, 
                            kernel=aff_kernel, random_state=rs)
        g_selects.fit(fc_selects)
        grad = g_selects.gradients_
        fscore = D(grad.mean(axis=0), grad)
        full_movie_grads[mname][_id] = fscore        

Compute ICC across movies using class as the ID

In [8]:
df = {'movieid': [], 'id': [], 'fscore': []}
df2 = {'movieid': [], 'id': [], 'fscore': []}

idx = 0
for mname in np.array(list(f_selected_gradients.keys())):
    for _id in f_selected_gradients[mname]:
        df['id'] += [int(_id[1:])]
        df['fscore'] += [f_selected_gradients[mname][_id][idx]]
        df['movieid'] += [mname]
        
        df2['id'] += [int(_id[1:])]
        df2['fscore'] += [full_movie_grads[mname][_id][idx]]
        df2['movieid'] += [mname]        

df = pd.DataFrame.from_dict(df)
df2 = pd.DataFrame.from_dict(df2)

icc = pingouin.intraclass_corr(data=df, targets='id', raters='movieid', ratings='fscore')
print(np.round(list(icc['ICC']),3))
icc2 = pingouin.intraclass_corr(data=df2, targets='id', raters='movieid', ratings='fscore')
print(np.round(list(icc2['ICC']),3))

[0.17  0.174 0.186 0.741 0.746 0.762]
[0.249 0.253 0.273 0.823 0.826 0.84 ]


In [13]:
for idx in range(0,414):
    df = {'movieid': [], 'id': [], 'fscore': []}
    df2 = {'movieid': [], 'id': [], 'fscore': []}
    for mname in f_selected_gradients:
        for _id in f_selected_gradients[mname]:
            df['id'] += [int(_id[1:])]
            df['fscore'] += [f_selected_gradients[mname][_id][idx]]
            df['movieid'] += [mname]

            df2['id'] += [int(_id[1:])]
            df2['fscore'] += [full_movie_grads[mname][_id][idx]]
            df2['movieid'] += [mname]        

    df = pd.DataFrame.from_dict(df)
    df2 = pd.DataFrame.from_dict(df2)

    print('vertex {}'.format(idx))
    icc = pingouin.intraclass_corr(data=df, targets='id', raters='movieid', ratings='fscore')
    print(np.round(list(icc['ICC']),3))
    icc2 = pingouin.intraclass_corr(data=df2, targets='movieid', raters='id', ratings='fscore')
    print(np.round(list(icc2['ICC']),3))

vertex 0
[0.206 0.211 0.23  0.784 0.789 0.807]
[0.075 0.083 0.113 0.709 0.731 0.793]
vertex 1
[0.121 0.124 0.131 0.658 0.665 0.678]
[0.052 0.056 0.065 0.62  0.641 0.677]
vertex 2
[0.296 0.3   0.327 0.855 0.857 0.872]
[0.051 0.062 0.092 0.619 0.664 0.753]
vertex 3
[0.173 0.177 0.19  0.745 0.75  0.766]
[0.039 0.047 0.062 0.551 0.597 0.664]
vertex 4
[0.263 0.267 0.289 0.833 0.836 0.85 ]
[0.04  0.05  0.073 0.553 0.611 0.703]
vertex 5
[0.312 0.316 0.345 0.864 0.866 0.881]
[0.045 0.056 0.087 0.586 0.642 0.741]
vertex 6
[0.208 0.21  0.217 0.786 0.788 0.795]
[0.025 0.032 0.041 0.431 0.497 0.561]
vertex 7
[0.33  0.332 0.347 0.873 0.875 0.882]
[0.012 0.024 0.037 0.272 0.423 0.534]
vertex 8
[0.323 0.327 0.355 0.87  0.872 0.885]
[0.038 0.05  0.081 0.545 0.615 0.725]
vertex 9
[0.199 0.203 0.218 0.776 0.781 0.796]
[0.043 0.052 0.072 0.574 0.622 0.701]
vertex 10
[0.28  0.283 0.304 0.845 0.847 0.859]
[0.036 0.047 0.072 0.53  0.598 0.7  ]
vertex 11
[0.303 0.307 0.337 0.859 0.861 0.877]
[0.064 0.074 0.1


KeyboardInterrupt

